<h2> Taller 5 - Cinemática Diferencial - Ejemplo robot planar</h2>
<br>Autor: Claudio Morales D.
<br>Centro de Automatización y Robótica Inacap
<br>Santiago, Chile, enero 2023

Este ejemplo es parte de la serie de actividades Robótica con Python, disponible en: <a>https://github.com/cmoralesd/robotica-con-python</a>
<br> Requiere la librería zmqRemoteAPI.
<br> Desde CoppeliaSim, abrir la escena 'robotPlanar.ttt'
<br> Los conceptos teóricos relacionados con esta actividad pueden revisarse en: <a>https://youtu.be/xMbIUCzFdR0</a>

In [ ]:
# importamos las librerías necesarias
import sympy

###  Conectamos con CoppeliaSim y obtenemos los manejadores
Utilizamos las funciones del API Remoto de CoppeliaSim. Para más detalles refiérase a la documentación de la librería:
<br> https://www.coppeliarobotics.com/helpFiles/en/apiFunctions.htm

In [ ]:
# Cargamos la librería, creamos un cliente y obtenemos acceso a sim
import zmqRemoteApi
client = zmqRemoteApi.RemoteAPIClient()
sim = client.getObject('sim')

# Obtenemos los manejadores para las articulaciones y el actuador final
j1 = sim.getObject('/joint1')
j2 = sim.getObject('/joint2')
dummy = sim.getObject('/Dummy')

print(j1, j2, dummy)

# ponemos el robot en posición inicial
sim.setJointTargetPosition(j1, 0)
sim.setJointTargetPosition(j2, 0)

<h3> 1. Obtenemos la cinemática directa</h3>
Conocidos los ángulos de las articulaciones, podemos conocer la posición final del actuador.

In [ ]:
# Comenzamos desde la descripción de Denavit-Hartenberg
#      theta     |      d      |      a      |    alpha
# ----------------------------------------------------------
#       q1             0             0.3            0
#       q2             0             0.2            0

In [ ]:
# Definimos una función para construir las matrices de transformación
# en forma simbóĺica a partir de los parámetros D-H

def symTfromDH(theta, d, a, alpha):
    # theta y alpha en radianes
    # d y a en metros
    Rz = sympy.Matrix([[sympy.cos(theta), -sympy.sin(theta), 0, 0],
                   [sympy.sin(theta), sympy.cos(theta), 0, 0],
                   [0, 0, 1, 0],
                   [0, 0, 0, 1]])
    tz = sympy.Matrix([[1, 0, 0, 0],
                   [0, 1, 0, 0],
                   [0, 0, 1, d],
                   [0, 0, 0, 1]])
    ta = sympy.Matrix([[1, 0, 0, a],
                   [0, 1, 0, 0],
                   [0, 0, 1, 0],
                   [0, 0, 0, 1]])
    Rx = sympy.Matrix([[1, 0, 0, 0],
                   [0, sympy.cos(alpha), -sympy.sin(alpha), 0],
                   [0, sympy.sin(alpha), sympy.cos(alpha), 0],
                   [0, 0, 0, 1]])
    T = Rz*tz*ta*Rx
    return T

In [ ]:
# La matriz de transformación desde la base al dummy es
q1 = sympy.symbols('q1')
q2 = sympy.symbols('q2')

T = sympy.simplify(symTfromDH(q1, 0, 0.3, 0)* symTfromDH(q2, 0, 0.2, 0))
T

<h3> 2. Situamos el robot en un punto mediante cinemática inversa</h3>

In [ ]:
# definimos las coordenadas de destino
a = 0.3
b = 0

# definimos las ecuaciones a resolver
ec1, ec2 = T[3]-a, T[7]-b
(ec1, ec2)

In [ ]:
# ahora resolvemos la ecuación utilizando nsolve()
try:
    q = sympy.nsolve((ec1, ec2),(q1,q2),(1,-1), prec=6)
except:
    print("no se pudo calcular")
    q = [0, 0, 0]
q

In [ ]:
# enviamos los ángulos al robot
sim.setJointTargetPosition(j1, float(q[0]))
sim.setJointTargetPosition(j2, float(q[1]))

<h3> 3. Y calculamos la cinemática diferencial en torno al punto de interés...</h3>

In [ ]:
# definimos la función f(q)
f = sympy.Matrix([T[3],T[7]])
f

In [ ]:
# obtenemos el jacobiano genérico
J = f.jacobian((q1, q2))
J

In [ ]:
# evaluamos el jaconiano en forma numérica para el punto de interés.
# Se debe usar la conversión a float() porque los elementos de q son matrices de sympy
val1 = float(q[0])
val2 = float(q[1])
J0 = J.evalf(subs={q1:val1, q2:val2})
J0

In [ ]:
# calculamos la inversa del jacobiano
Jinv = J0.inv()
Jinv

In [ ]:
# definimos una magnitud de desplazamiento dx, luego calculamos dq = Jinv * dx
dx = sympy.Matrix([0.02, 0])
dx

In [ ]:
dq = Jinv * dx
dq

In [ ]:
# finalmente enviamos los nuevos ángulos al robot
sim.setJointTargetPosition(j1, float(q[0] + dq[0]))
sim.setJointTargetPosition(j2, float(q[1] + dq[1]))